In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import datetime
from urllib.request import urlopen, Request

In [2]:
r = Request('https://th.investing.com/economic-calendar/', headers={'User-Agent': 'Mozilla/5.0'})
time.sleep(2)
response = urlopen(r).read()
soup = BeautifulSoup(response, "html.parser")
table = soup.find_all(class_ = "js-event-item")
result = []

In [3]:
def event_calendar():
    
    for bl in table:
        event_datetime = bl.get('data-event-datetime', '').strip()
        currency = bl.find(class_="left flagCur noWrap").text.strip()
        intensity_divs = bl.find_all(class_="left textNum sentiment noWrap")
        event = bl.find(class_="left event").text.strip()
        intencity_val = 0
        true_count = 0

        for intence in intensity_divs:
            _true = intence.find_all(class_="grayFullBullishIcon")
            _false = intence.find_all(class_="grayEmptyBullishIcon")

            true_count = len(_true)

            if true_count == 3:
                intencity_val = 3
            elif true_count == 2:
                intencity_val = 2
            else :
                intencity_val = 1
                
        event_datetime = event_datetime.split(' ')
        date=event_datetime[0]
        time=event_datetime[1]
        
        
        result.append({'currency' : currency, 'date' : date, 'time' : time, 'intensity' : intencity_val, 'event':event})

    return result

In [4]:
news = event_calendar()
news_df = pd.DataFrame(news)

In [5]:
display(news_df.shape)
display(news_df.head(10))

(39, 5)

,currency,date,time,intensity,event
0,USD,2025/06/17,00:00:00,2,การประมูลพันธบัตรอายุ 20 ปี
1,KRW,2025/06/17,04:00:00,1,ราคาส่งออก (ปีต่อปี) ( พ.ค.)
2,KRW,2025/06/17,04:00:00,1,ดัชนีราคาสินค้านำเข้า (ปีต่อปี) ( พ.ค.)
3,NZD,2025/06/17,05:45:00,1,ดัชนีราคาอาหาร (FPI) (เดือนต่อเดือน) ( พ.ค.)
4,SGD,2025/06/17,07:30:00,1,ปริมาณการส่งออกสินค้าในประเทศทั่วไปยกเว้นน้ำมั...
5,SGD,2025/06/17,07:30:00,1,ปริมาณการส่งออกสินค้าในประเทศทั่วไปยกเว้นน้ำมั...
6,SGD,2025/06/17,08:00:00,1,ดุลการค้าของสิงคโปร์ ( พ.ค.)
7,JPY,2025/06/17,09:30:00,2,การแถลงการณ์นโยบายการเงินจากธนาคารกลางญี่ปุ่น
8,JPY,2025/06/17,10:00:00,3,การตัดสินใจเกี่ยวกับอัตราดอกเบี้ย
9,SEK,2025/06/17,13:00:00,1,อัตราการว่างงานของสวีเดน ( พ.ค.)


In [6]:
now = datetime.datetime.now()
date_time = now.strftime("%Y-%m-%d %H-%M-%S").strip().replace(' ', '_')
news_df.to_csv(f'Data/Event_Calendar/{date_time}.csv', index=False)

In [7]:
print(news_df[news_df['intensity'] == 3][['currency', 'event']])

   currency                                              event
8       JPY                  การตัดสินใจเกี่ยวกับอัตราดอกเบี้ย
18      USD  ดัชนียอดขายปลีกพื้นฐาน (Core Retail Sales) (เด...
24      USD           ดัชนียอดขายปลีก (เดือนต่อเดือน)  ( พ.ค.)


In [8]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

load_dotenv("../SentimentAnalysis/GPT/secret.env")

mongo_connection_string = os.getenv("MONGO_CONNECTION_STRING")

try:
    client = MongoClient(mongo_connection_string)
    db = client['stock_news_db']
    collection = db['event_calendar']

    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")

except Exception as e:
    print(f"An error occurred: {e}")

Pinged your deployment. You successfully connected to MongoDB!


In [9]:
complete_dict=news_df.to_dict(orient='records')

result = collection.insert_many(complete_dict,ordered=True)
print(f"Successfully inserted document with id: {result.inserted_ids}")

Successfully inserted document with id: [ObjectId('68513fdc193036329a06089e'), ObjectId('68513fdc193036329a06089f'), ObjectId('68513fdc193036329a0608a0'), ObjectId('68513fdc193036329a0608a1'), ObjectId('68513fdc193036329a0608a2'), ObjectId('68513fdc193036329a0608a3'), ObjectId('68513fdc193036329a0608a4'), ObjectId('68513fdc193036329a0608a5'), ObjectId('68513fdc193036329a0608a6'), ObjectId('68513fdc193036329a0608a7'), ObjectId('68513fdc193036329a0608a8'), ObjectId('68513fdc193036329a0608a9'), ObjectId('68513fdc193036329a0608aa'), ObjectId('68513fdc193036329a0608ab'), ObjectId('68513fdc193036329a0608ac'), ObjectId('68513fdc193036329a0608ad'), ObjectId('68513fdc193036329a0608ae'), ObjectId('68513fdc193036329a0608af'), ObjectId('68513fdc193036329a0608b0'), ObjectId('68513fdc193036329a0608b1'), ObjectId('68513fdc193036329a0608b2'), ObjectId('68513fdc193036329a0608b3'), ObjectId('68513fdc193036329a0608b4'), ObjectId('68513fdc193036329a0608b5'), ObjectId('68513fdc193036329a0608b6'), ObjectId(